# H2-stretch
In this notebook, the previous coded implementation of RHF and UHF will be visualised using $H_{2}$ as test-molecule.

In [7]:
import matplotlib as plt
import numpy as np
from pyscf import gto, scf
import scipy

In [22]:
MAXITER = 40
E_conv = 1.0e-6

In [23]:
distance = np.linspace(1.0,8.0, 1000)
mol = gto.M(atom = 'H 0.0 0.0 0.0; H 1.0 0.0 0.0', basis = 'ccpvdz')
S = mol.intor('int1e_ovlp')
T = mol.intor('int1e_kin')
V = mol.intor('int1e_nuc')
H_core = T + V
eri = mol.intor('int2e')

In [17]:
# ==> SCF Iterations <==
# Pre-iteration energy declarations
SCF_E_UHF = 0.0
E_old = 0.0

print('==> Starting SCF Iterations <==\n')
eps, C = scipy.linalg.eigh(H_core, S)
n_alpha = mol.nelec[0]
n_beta = mol.nelec[1]

Calpha = C[:, :n_alpha]
Cbeta = C[:, :n_beta]


D_alpha = np.einsum('pi,qi->pq', Calpha, Calpha, optimize=True)
D_beta = np.einsum('pi,qi->pq', Cbeta, Cbeta, optimize=True)
D = np.einsum('pi,qi->pq', C[:,:max(n_alpha, n_beta)], C[:,:max(n_alpha, n_beta)], optimize=True)

# Begin Iterations
for scf_iter in range(1, MAXITER + 1):
    # Build Fock matrices: F_alpha = H + (J - K)_alpha + J_beta and F_beta = H + (J - K)_beta + J_alpha
    # The same procedure will be used but there will be a separation between the F_alpha and F_beta
    J_alpha = np.einsum('pqrs,rs->pq', eri, D_alpha, optimize=True)
    K_alpha = np.einsum('prqs,rs->pq', eri, D_alpha, optimize=True)
    J_beta = np.einsum('pqrs,rs->pq', eri, D_beta, optimize=True)
    K_beta = np.einsum('prqs,rs->pq', eri, D_beta, optimize=True)


    F_alpha = (J_alpha - K_alpha  + J_beta) + H_core
    F_beta = (J_alpha + J_beta - K_beta) + H_core
    F = H_core + (J_alpha - K_alpha) + (J_beta - K_beta)
    # Compute RHF energy: BO nuclear energy and RHF electronc energy
    enuc = mol.get_enuc() # The BO is not different from RHF
    #E_UHF_el_matrix_alpha = ( H_core + F_alpha) * D_alpha
    #E_UHF_el_matrix_beta = (F_beta + H_core) * D_beta
    E_UHF_matrix = 0.5 * (D_alpha * (F_alpha + H_core) +  D_beta * (F_beta + H_core))
    #E_UHF_el = np.sum(E_UHF_el_matrix_alpha) + np.sum(E_UHF_el_matrix_beta)
    E_UHF_el = np.sum(E_UHF_matrix)
    SCF_E_UHF = enuc + E_UHF_el

    # SCF Converged?
    if (abs(SCF_E_UHF - E_old) < E_conv):
        break
    E_old = SCF_E_UHF
    
    # Compute new orbital guess
    eps, C1 = scipy.linalg.eigh(F_alpha,S)
    eps1 , C2 = scipy.linalg.eigh(F_beta,S)

    Calpha = C1[:, :n_alpha]
    Cbeta = C2[:, : n_beta]

    #D = np.einsum('pi,qi->pq', C[:,:max(n_alpha, n_beta)], C[:,:max(n_alpha, n_beta)], optimize=True)
    D_alpha = np.einsum('pi,qi->pq', Calpha, Calpha, optimize=True)
    D_beta = np.einsum('pi,qi->pq', Cbeta, Cbeta, optimize=True)
    #D = np.einsum('pi,qi->pq', C[:,:(n_alpha + n_beta)], C[:,:(n_alpha + n_beta)], optimize=True)

    # MAXITER exceeded?
    if (scf_iter == MAXITER):
        print(SCF_E_UHF)
        raise Exception("Maximum number of SCF iterations exceeded.")


# Post iterations
print('\nSCF converged.')
print('Final UHF Energy: %.8f [Eh]' % (SCF_E_UHF))

==> Starting SCF Iterations <==


SCF converged.
Final UHF Energy: -1.10015376 [Eh]


In [ ]:
energy_UHF = SCF_E_UHF
energy_RHF = SCF_E_RHF
plt.plot()